In [1]:
!pip install PyPDF2

     |████████████████████████████████| 77 kB 2.3 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=6bc4088a9c3cdf764fb61df2f359c7fc911fdffbdacfb2a93f233c396a15c5ba
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [2]:
!pip install bs4

In [3]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=d7307464b93e523e3072b3ec72b68fb577f9c8ea385f95f1171489f24c74b394
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [4]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 958 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 44.6 MB/s 
     |████████████████████████████████| 356 kB 47.0 MB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 3.6 MB 47.0 MB/s 
     |████████████████████████████████| 54 kB 1.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Basic libraries
import pandas as pd
import numpy as np
import nltk

# Importing for scraping
from bs4 import BeautifulSoup 
import fake_useragent

import random
import time         # Tracking time
import requests     # HTTP requests
import re           # String manipulation

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [7]:
import requests
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = []
    for i in parser.xpath('//tbody/tr'):
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.append(proxy)
    return proxies

In [8]:
def download_file(download_url,name, proxy):
    response = requests.get(download_url, proxies={"http": proxy, "https": proxy} )
    
    with open(f'{name}', 'wb') as f:
        f.write(response.content)
    f.close()

In [9]:
import PyPDF2

In [10]:
def pdf2str(pdf_link, fake_proxy):
  
    if fake_proxy:
        i = 0
        while True:
            proxy = get_proxies()[i]
            try:
                download_file(pdf_link, 'sentenza.pdf', proxy)
                break
            except:
                i += 1
    else:
        time.sleep(random.randint(1,5))
        !wget '{pdf_link}'  -O 'sentenza.pdf'
    pdfFileObj = open('sentenza.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    text = ''
    for p in range(pdfReader.numPages):
        text +=  pdfReader.getPage(p).extractText()
    !rm 'sentenza.pdf'
    return text

In [11]:
from urllib.parse import unquote

In [12]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [13]:
links = []

driver.get('http://www.italgiure.giustizia.it/sncass/')
time.sleep(random.randint(1,5))
driver.find_element_by_id('2.[kind]').click()
for p in range(30):
    time.sleep(random.randint(1,5))
    cards = driver.find_elements_by_class_name('card')
    new_links = ['http://www.italgiure.giustizia.it' + unquote(x.find_elements_by_css_selector('a')[1].find_elements_by_css_selector('span')[0].find_element_by_css_selector('span').get_attribute('data-arg')) for x in cards]
    try:
        links.extend(new_links)
    except:
        pass
    time.sleep(random.randint(3,5))
    driver.find_element_by_class_name('pages').find_element_by_css_selector('span').click()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_* commands are depreca

In [14]:
len(links)

300

In [15]:
fake_ip = False
i = 0
texts = []
for l in links:
    
    try:     
        texts.append(pdf2str(l, fake_proxy = fake_ip))
    except:
        fake_ip = True
        
    print('ok: ', i)
    i += 1

        

    

--2021-12-30 18:47:07--  http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211223/snpen@s20@a2021@n47125@tS.clean.pdf
Resolving www.italgiure.giustizia.it (www.italgiure.giustizia.it)... 89.119.251.144
Connecting to www.italgiure.giustizia.it (www.italgiure.giustizia.it)|89.119.251.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457976 (447K) [application/pdf]
Saving to: ‘sentenza.pdf’

sentenza.pdf        100%[===================>] 447.24K   152KB/s    in 2.9s    

2021-12-30 18:47:10 (152 KB/s) - ‘sentenza.pdf’ saved [457976/457976]

ok:  0
--2021-12-30 18:47:15--  http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211223/snpen@s10@a2021@n47058@tO.clean.pdf
Resolving www.italgiure.giustizia.it (www.italgiure.giustizia.it)... 89.119.251.144
Connecting to www.italgiure.giustizia.it (www.italgiure.giustizia.it)|89.119.251.144|:80... connected.
HTTP request sent, awai

In [16]:
df_sentenze = pd.DataFrame(
                            {'link': links,
                            
                            'text': texts
                            }
                           )

ValueError: ignored

In [17]:
len(links)

300

In [18]:
len(texts)

178

In [ ]:
df_sentenze.drop_duplicates(subset='text',inplace= True)

In [ ]:
df_sentenze.to_csv('/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization/data/sentenze_pdf.csv',index = False)

In [ ]:
df_sentenze

In [ ]:
def scrape_1_page(driver):
    
    time.sleep(random.randint(1,5))
    driver.find_element_by_id('2.[kind]').click()
    time.sleep(random.randint(1,5))
    cards = driver.find_elements_by_class_name('card')
    links = ['http://www.italgiure.giustizia.it' + unquote(x.find_elements_by_css_selector('a')[1].find_elements_by_css_selector('span')[0].find_element_by_css_selector('span').get_attribute('data-arg')) for x in cards]

    return links

In [ ]:
driver.get('http://www.italgiure.giustizia.it/sncass/')
scrape_1_page(driver)

['http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s60@a2021@n47001@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s60@a2021@n47000@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s60@a2021@n46999@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46852@tS.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46851@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46841@tS.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46840@tS.clea

In [ ]:
driver.get('http://www.italgiure.giustizia.it/sncass/')
links = scrape_1_page(driver)
for p in range(9):
    driver.get('http://www.italgiure.giustizia.it/sncass/')
    time.sleep(random.randint(1,5))
    driver.find_element_by_id('2.[kind]').click()
    time.sleep(random.randint(1,5))
    driver.find_element_by_class_name('pages').find_elements_by_css_selector('a')[p].click()
    time.sleep(random.randint(1,5))
    cards = driver.find_elements_by_class_name('card')
    links.extend(['http://www.italgiure.giustizia.it' + unquote(x.find_elements_by_css_selector('a')[1].find_elements_by_css_selector('span')[0].find_element_by_css_selector('span').get_attribute('data-arg')) for x in cards])

In [ ]:
len(links)

100

In [ ]:
len(set(links))

100

In [ ]:
#links

['http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s60@a2021@n47001@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s60@a2021@n47000@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s60@a2021@n46999@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46852@tS.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46851@tO.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46841@tS.clean.pdf',
 'http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snpen&id=./20211222/snpen@s40@a2021@n46840@tS.clea

In [ ]:

texts = []

for link in links:
    try:
        texts.append(pdf2str(link))
    except:
        texts.append('pdf error')

df_sentenze = pd.DataFrame(
                            {'link': links,
                            
                            'text': texts
                            }
                           )


In [ ]:
df_sentenze['text'].nunique()

47

In [ ]:
pdf2str('http://www.italgiure.giustizia.it/xway/application/nif/clean/hc.dll?verbo=attach&db=snciv&id=./20211220/snciv@sL0@a2021@n40904@tO.clean.pdf')

PdfReadError: ignored